In [1]:
!pip install qiskit qiskit-aer matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00


In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def inverse_qft(circuit, qubits):
    """Applies inverse QFT on specified qubits."""
    n = len(qubits)
    for j in range(n // 2):
        circuit.swap(qubits[j], qubits[n - j - 1])
    for j in range(n):
        for m in range(j):
            circuit.cp(-np.pi / (2 ** (j - m)), qubits[m], qubits[j])
        circuit.h(qubits[j])
    return circuit

In [4]:
def qpe(theta=0.125, n_count=3):
    """
    Quantum Phase Estimation demonstration
    theta: phase to estimate (0 ≤ theta < 1)
    n_count: number of counting qubits (precision)
    """
    qc = QuantumCircuit(n_count + 1, n_count)
    count_qubits = range(n_count)
    target = n_count

    # Step 1: Apply Hadamard gates to counting qubits
    qc.h(count_qubits)

    # Step 2: Prepare target qubit in eigenstate |1>
    qc.x(target)

    # Step 3: Controlled unitary operations (controlled phase rotations)
    for q in range(n_count):
        qc.cp(2 * np.pi * theta * (2 ** q), count_qubits[q], target)

    # Step 4: Apply inverse QFT on counting qubits
    inverse_qft(qc, count_qubits)

    # Step 5: Measure counting qubits
    qc.measure(count_qubits, range(n_count))

    return qc

In [6]:
theta = 0.375   # Try 0.25, 0.5, etc.
n_count = 4     # Increase for higher precision

qc = qpe(theta, n_count)

# Visualize the circuit safely
try:
    qc.draw('mpl')
    plt.show()
except Exception as e:
    print(f"Drawing failed: {e}")
    print(qc.draw())  # fallback to text mode

# Simulate using Aer
sim = AerSimulator()
t_qc = transpile(qc, sim)
job = sim.run(t_qc, shots=2048)
result = job.result()
counts = result.get_counts()

# Display results
print("Measurement Results:", counts)
plot_histogram(counts)
plt.show()

Drawing failed: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."
     ┌───┐                                       ┌───┐                         »
q_0: ┤ H ├─■───────────────────────────────────X─┤ H ├─■──────────────■────────»
     ├───┤ │                                   │ └───┘ │P(-π/2) ┌───┐ │        »
q_1: ┤ H ├─┼─────────■───────────────────X─────┼───────■────────┤ H ├─┼────────»
     ├───┤ │         │                   │     │                └───┘ │P(-π/4) »
q_2: ┤ H ├─┼─────────┼─────────■─────────X─────┼──────────────────────■────────»
     ├───┤ │         │         │               │                               »
q_3: ┤ H ├─┼─────────┼─────────┼───────■───────X───────────────────────────────»
     ├───┤ │P(3π/4)  │P(3π/2)  │P(3π)  │P(6π)                                  »
q_4: ┤ X ├─■─────────■─────────■───────■───────────────────────────────────────»
     └───┘                                                

In [7]:
# ------------------------------------------------------
# 4. Theoretical Phase Comparison
# ------------------------------------------------------
phase_decimal = theta
phase_binary = format(int(phase_decimal * (2 ** n_count)), f'0{n_count}b')
print(f"Theoretical phase (decimal): {phase_decimal}")
print(f"Expected binary output: {phase_binary}")

Theoretical phase (decimal): 0.375
Expected binary output: 0110


In [8]:
# 5. Optional: Noise Simulation (Advanced)
# ------------------------------------------------------
from qiskit_aer.noise import NoiseModel, depolarizing_error

# Create a simple depolarizing noise model
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.01, 1), ['u3'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.01, 2), ['cx'])

job_noisy = sim.run(t_qc, shots=2048, noise_model=noise_model)
result_noisy = job_noisy.result()
counts_noisy = result_noisy.get_counts()

print("\nWith Noise Model:")
plot_histogram([counts, counts_noisy], legend=["Ideal", "Noisy"])
plt.show()


With Noise Model:
